# Efficiency: making programs multi-threaded
_COSC 208, Introduction to Computer Systems, 2023-11-29_

## Announcements
* Project 4 due Thurs, Dec 7

## Outline
* Passing multiple parameters to threads
* Transforming programs to be multi-threaded

## Passing multiple parameters to threads

* A thread can only take a single `void *` pointer as a parameter
* To pass multiple parameters:
    * Define a `struct` that contains a field for each of the parameters
    * Allocate space for an instance of the `struct`
    * Pass a pointer to the instance to the thread — i.e., as the last parameter in `pthread_create`

## Transforming programs to be multi-threaded

* For-each style loops are natural candidates for threading
    * Example: a program that counts the number of occurrences of each vowel within a string
    * Another example: a program that counts the longest sequence of each Short Tandem Repeat (STR) in a DNA sequence
* Steps
    * Move the body of the loop into its own "worker" function
    * Write a "wrapper" function which:
        * Has the required signature for a thread function (i.e., takes a single `void *` parameter and returns a void pointer)
        * Extracts the argument(s) for the "worker" function from the thread function's arguments
        * Calls the "worker" function
        * Prepares the return value from the "worker" function to be returned by the thread
    * Replace the body of the loop with the creation of threads that execute the "wrapper" function
    * Add a second loop to wait for all threads to complete

* _Assume you are given the following code:_

In [ ]:
#include <ctype.h>
#include <stdio.h>
#include <string.h>
int count(char *str, char ch) {
    int num = 0;
    while (*str != '\0') {
        if (tolower(*str) == ch) {
            num++;
        }
        str++;
    }
    return num;
}
int main(int argc, char *argv[]) {
    if (argc < 2) {
        printf("Error: provide a string\n");
        return 1;
    }
    char *str = argv[1];
    char *vowels = "aeiou";
    int counts[strlen(vowels)];
    for (int i = 0; i < strlen(vowels); i++) {
        counts[i] = count(str, vowels[i]);
    }

    for (int i = 0; i < strlen(vowels); i++) {
        printf("%c %d\n", vowels[i], counts[i]);
    }
}

* Q4: _Write a function called  `count_wrapper` that has the signature required for a thread function and calls the `count` function. (Hint: you'll need to declare a `struct` that contains all of the parameters required for `count`.)_

* Q5: _Re-write `main` to create/wait for threads that execute `count_wrapper` (instead of calling `count` sequentially)._

In [ ]:
#include <ctype.h>
#include <stdio.h>
#include <string.h>
#include <pthread.h>
int count(char *str, char ch) {
    int num = 0;
    while (*str != '\0') {
        if (tolower(*str) == ch) {
            num++;
        }
        str++;
    }
    return num;
}
struct count_args {
    char *str;
    char ch;
};
void *count_wrapper(void *arg) {
    struct count_args *cargs = (struct count_args *)arg;
    int *result = malloc(sizeof(int));
    *result = count(cargs->str, cargs->ch);
    return result;
}
int main(int argc, char *argv[]) {
    if (argc < 2) {
        printf("Error: provide a string\n");
        return 1;
    }
    char *str = argv[1];
    char *vowels = "aeiou";

    // Create threads
    struct count_args args[strlen(vowels)];
    pthread_t threads[strlen(vowels)];
    for (int i = 0; i < strlen(vowels); i++) {
        args[i].str = str;
        args[i].ch = vowels[i];
        pthread_create(&(threads[i]), NULL, &count_wrapper, &(args[i]));
    }

    // Wait for threads
    for (int i = 0; i < strlen(vowels); i++) {
        int *ptr;
        pthread_join(threads[i], (void **)(&ptr));
        printf("%c %d\n", vowels[i], *ptr);
    }
}